# yelp data ingestion

see example: https://github.com/polakowo/yelp-3nf

In [2]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Window, Row

import pprint
import boto3
import json
import os

pp = pprint.PrettyPrinter(indent = 3)
print('imported modules.')

imported modules.


## connect to s3

In [3]:
client = boto3.client('s3')

bucket_meta = client.list_objects(Bucket = 'yelp-dataset-stevenhurwitt')
print('files in s3 bucket:')
print('')
for c in bucket_meta['Contents']:
    print(c['Key'])

files in s3 bucket:

raw/
raw/yelp_academic_dataset_business.json
raw/yelp_academic_dataset_checkin.json
raw/yelp_academic_dataset_review.json
raw/yelp_academic_dataset_tip.json
raw/yelp_academic_dataset_user.json


## function to read json from s3

In [4]:
def read_json(filename):
    """
    reads a yelp .json file from s3 bucket.

    keyword arguments:
    filename - name of file (str)

    returns: json_file (json)
    """

    bucket = "yelp-dataset-stevenhurwitt"
    print(f"bucket: {bucket}.")
    print(f"filename: {filename}.")
    
    response = client.get_object(Bucket = bucket, Key = filename)
    file_content = response['Body'].read().decode('utf-8')
    json_file = json.loads("[" + file_content.replace("}\n{", "},\n{") + "]")
    return(json_file)

In [5]:
my_file = read_json('raw/yelp_academic_dataset_business.json')
my_file

bucket: yelp-dataset-stevenhurwitt.
filename: raw/yelp_academic_dataset_business.json.


[{'business_id': 'Pns2l4eNsfO8kk83dixA6A',
  'name': 'Abby Rappoport, LAC, CMQ',
  'address': '1616 Chapala St, Ste 2',
  'city': 'Santa Barbara',
  'state': 'CA',
  'postal_code': '93101',
  'latitude': 34.4266787,
  'longitude': -119.7111968,
  'stars': 5.0,
  'review_count': 7,
  'is_open': 0,
  'attributes': {'ByAppointmentOnly': 'True'},
  'categories': 'Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists',
  'hours': None},
 {'business_id': 'mpf3x-BjTdTEA3yCZrAYPw',
  'name': 'The UPS Store',
  'address': '87 Grasso Plaza Shopping Center',
  'city': 'Affton',
  'state': 'MO',
  'postal_code': '63123',
  'latitude': 38.551126,
  'longitude': -90.335695,
  'stars': 3.0,
  'review_count': 15,
  'is_open': 1,
  'attributes': {'BusinessAcceptsCreditCards': 'True'},
  'categories': 'Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services',
  'hours': {'Monday': '0:0-0:0',
   'Tuesday': '8:0-18:30',
   'Wednes

In [8]:
# pp.pprint(my_file[0])

In [7]:
pp.pprint(my_file[3])

{  'address': '935 Race St',
   'attributes': {  'Alcohol': "u'none'",
                    'BikeParking': 'True',
                    'BusinessAcceptsCreditCards': 'False',
                    'BusinessParking': "{'garage': False, 'street': True, "
                                       "'validated': False, 'lot': False, "
                                       "'valet': False}",
                    'ByAppointmentOnly': 'False',
                    'Caters': 'True',
                    'OutdoorSeating': 'False',
                    'RestaurantsDelivery': 'False',
                    'RestaurantsPriceRange2': '1',
                    'RestaurantsTakeOut': 'True',
                    'WiFi': "u'free'"},
   'business_id': 'MTSW4McQd7CbVtyjqoe9mw',
   'categories': 'Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries',
   'city': 'Philadelphia',
   'hours': {  'Friday': '7:0-21:0',
               'Monday': '7:0-20:0',
               'Saturday': '7:0-21:0',
               'Sunday': '7:0-2

## create dynamodb yelp-checkin table

In [ ]:
# yelp_business.delete()

NameError: name 'checkin_table' is not defined

In [ ]:
dynamodb = boto3.resource('dynamodb', endpoint_url="https://us-east-2.console.aws.amazon.com?arn=arn:aws:dynamodb:us-east-2134132211607:8000")

In [ ]:
yelp_business = dynamodb.create_table(
        TableName='yelp.business',
        KeySchema=[
            {
                'AttributeName': 'business_id',
                'KeyType': 'HASH'  # Partition key
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'name',
                'AttributeType': 'S'
            }
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 25,
            'WriteCapacityUnits': 20
        }
    )
print('created dynamo table.')

ResourceInUseException: An error occurred (ResourceInUseException) when calling the CreateTable operation: Cannot create preexisting table

## checkin

In [ ]:
checkin = read_json('s3a://yelp-dataset-stevenhurwitt/yelp_academic_dataset_checkin.json')
print('read in json file.')

read in json file.


In [ ]:
print('json file has {} records with size of {} mb.'.format(len(checkin), sys.getsizeof(checkin)/1000000))
print('here is an example record:')
print('')
pp.pprint(checkin[0])

json file has 138876 records with size of 1.140568 mb.
here is an example record:

{  'business_id': '--0zrn43LEaB4jUWTQH_Bg',
   'date': '2010-10-08 22:21:20, 2010-11-01 21:29:14, 2010-12-23 22:55:45, '
           '2011-04-08 17:14:59, 2011-04-11 21:28:45, 2011-04-26 16:42:25, '
           '2011-05-20 19:30:57, 2011-05-24 20:02:21, 2011-08-29 19:01:31'}


## insert checkin items into dynamo table

In [ ]:
# checkin_missing_index = []

# for i, c in enumerate(checkin):
#     try:
#         checkin_table.put_item(Item = c)
#     except:
#         print('item too large, index {}, skipping...'.format(i))
#         checkin_missing_index.append(i)

# print('loaded data into table.')

index 138400, skipping...
item too large, index 138401, skipping...
item too large, index 138402, skipping...
item too large, index 138403, skipping...
item too large, index 138404, skipping...
item too large, index 138405, skipping...
item too large, index 138406, skipping...
item too large, index 138407, skipping...
item too large, index 138408, skipping...
item too large, index 138409, skipping...
item too large, index 138410, skipping...
item too large, index 138411, skipping...
item too large, index 138412, skipping...
item too large, index 138413, skipping...
item too large, index 138414, skipping...
item too large, index 138415, skipping...
item too large, index 138416, skipping...
item too large, index 138417, skipping...
item too large, index 138418, skipping...
item too large, index 138419, skipping...
item too large, index 138420, skipping...
item too large, index 138421, skipping...
item too large, index 138422, skipping...
item too large, index 138423, skipping...
item too

In [ ]:
# print(checkin_table.item_count)

NameError: name 'checkin_table' is not defined

## business

In [ ]:
business = read_json('s3a://yelp-dataset-stevenhurwitt/yelp_academic_dataset_business.json')
print('read in json file.')

read in json file.


In [ ]:
print('json file has {} records with size of {} mb.'.format(len(business), sys.getsizeof(business)/1000000))
print('here is an example record:')
print('')
pp.pprint(business[0])

json file has 160585 records with size of 1.443576 gb.
here is an example record:

{  'address': '921 Pearl St',
   'attributes': {  'Alcohol': "'beer_and_wine'",
                    'Ambience': "{'touristy': False, 'hipster': False, "
                                "'romantic': False, 'divey': False, "
                                "'intimate': False, 'trendy': False, "
                                "'upscale': False, 'classy': False, 'casual': "
                                'True}',
                    'BikeParking': 'True',
                    'BusinessAcceptsBitcoin': 'False',
                    'BusinessAcceptsCreditCards': 'True',
                    'BusinessParking': "{'garage': False, 'street': True, "
                                       "'validated': False, 'lot': False, "
                                       "'valet': False}",
                    'Caters': 'True',
                    'DogsAllowed': 'False',
                    'GoodForMeal': "{'dessert': False,

In [ ]:
del business

In [ ]:
checkin[0]['date']

str

In [ ]:
# for i in range(1,len(checkin)):
    # mongo_upload(checkin[i])
    # pp.pprint(checkin[i])

# print('uploaded checkin collection.')

OperationFailure: Fully qualified namespace is too long. Namespace: yelp.{'business_id': '--0zrn43LEaB4jUWTQH_Bg', 'date': '2010-10-08 22:21:20, 2010-11-01 21:29:14, 2010-12-23 22:55:45, 2011-04-08 17:14:59, 2011-04-11 21:28:45, 2011-04-26 16:42:25, 2011-05-20 19:30:57, 2011-05-24 20:02:21, 2011-08-29 19:01:31', '_id': ObjectId('60691a52c189385debb59268')} Max: 255, full error: {'ok': 0.0, 'errmsg': "Fully qualified namespace is too long. Namespace: yelp.{'business_id': '--0zrn43LEaB4jUWTQH_Bg', 'date': '2010-10-08 22:21:20, 2010-11-01 21:29:14, 2010-12-23 22:55:45, 2011-04-08 17:14:59, 2011-04-11 21:28:45, 2011-04-26 16:42:25, 2011-05-20 19:30:57, 2011-05-24 20:02:21, 2011-08-29 19:01:31', '_id': ObjectId('60691a52c189385debb59268')} Max: 255", 'code': 4862100, 'codeName': 'Location4862100'}

In [ ]:
# str(business[0]["business_id"])

OperationFailure: Fully qualified namespace is too long. Namespace: yelp.{'business_id': '--164t1nclzzmca7eDiJMw', 'date': '2010-02-26 02:06:53, 2010-02-27 08:00:09, 2010-03-04 02:00:59, 2010-03-11 01:24:46, 2010-03-17 02:29:17, 2010-03-19 03:33:40, 2010-03-27 01:26:31, 2010-04-07 20:08:52, 2010-04-12 19:03:33, 2010-04-27 18:15:59, 2010-04-29 17:59:15, 2010-05-16 00:28:06, 2010-05-18 17:51:22, 2010-05-22 03:25:43, 2010-05-22 04:15:07, 2010-05-23 06:42:25, 2010-06-04 04:59:14, 2010-06-10 17:33:19, 2010-06-20 05:48:30, 2010-06-25 23:32:03, 2010-07-09 19:18:13, 2010-07-10 01:58:37, 2010-07-11 03:00:13, 2010-07-14 17:47:55, 2010-07-16 19:14:11, 2010-07-17 01:33:04, 2010-07-17 01:48:42, 2010-07-18 02:37:39, 2010-07-18 03:51:35, 2010-07-22 19:20:00, 2010-07-24 01:46:18, 2010-07-25 03:45:09, 2010-07-30 01:07:12, 2010-08-01 05:31:38, 2010-08-03 19:05:25, 2010-08-05 01:37:29, 2010-08-05 23:00:52, 2010-08-07 22:50:59, 2010-08-12 02:10:44, 2010-08-15 03:29:10, 2010-08-21 00:54:52, 2010-08-22 03:3

In [ ]:
str(business[0]["date"])

In [ ]:
try:
    pp.pprint(business[0])

except KeyboardInterrupt:
    logger.info(KeyboardInterrupt)
    logger.info(business[1])

:26, '
              '2020-08-31 18:51:28, 2020-11-23 19:01:21, 2020-11-27 18:11:52, '
              '2020-12-28 20:36:41, 2020-12-28 20:37:52, 2021-01-01 16:51:04'},
   {  '_id': ObjectId('604f9773c4078d81432ffd50'),
      'business_id': 'G_QFvmWWmJRC-bsWC4bYqw',
      'date': '2010-07-21 23:13:40, 2010-08-03 20:16:48, 2011-01-21 23:33:30, '
              '2011-04-26 18:27:07, 2011-07-20 17:31:37, 2011-07-22 19:51:18, '
              '2012-03-25 01:08:27, 2012-04-14 18:34:58, 2013-01-01 14:53:20, '
              '2014-07-26 18:09:26, 2015-03-26 23:26:34, 2015-09-02 12:14:57, '
              '2015-09-03 22:02:55, 2015-09-15 13:06:04, 2015-11-23 12:47:22, '
              '2015-11-30 13:48:01, 2016-02-04 12:34:50, 2016-02-11 12:26:13, '
              '2016-02-17 14:16:28, 2016-02-29 14:30:04, 2016-03-01 12:33:33, '
              '2016-03-16 12:01:27, 2016-03-18 21:57:52, 2016-03-22 11:23:42, '
              '2017-01-09 13:24:04, 2017-01-14 14:24:44, 2017-02-07 12:00:09, '
              '

KeyboardInterrupt: 

In [ ]:
del checkin

## tip

In [ ]:
tip = read_json('s3a://yelp-dataset-stevenhurwitt/yelp_academic_dataset_tip.json')
logger.info('read in json file.')

read in json file.


In [ ]:
logger.info('json file has {} records with size of {} mb.'.format(len(tip), sys.getsizeof(tip)/1000000))
logger.info('here is an example record:')
logger.info('')
logger.info(tip[0])

json file has 1162119 records with size of 9.504856 gb.
here is an example record:

{  'business_id': 'ENwBByjpoa5Gg7tKgxqwLg',
   'compliment_count': 0,
   'date': '2011-07-22 19:07:35',
   'text': 'Carne asada chips...',
   'user_id': 'WCjg0jdHXMlwbqS9tZUx8Q'}


In [ ]:
del tip

## user

In [ ]:
try:
    user = read_json('s3a://yelp-dataset-stevenhurwitt/yelp_academic_dataset_user.json')
    print('read in json file.')

except Exception as e:
    logger.warn(e)

OverflowError: signed integer is greater than maximum

In [ ]:
logger.info('json file has {} records with size of {} mb.'.format(len(user), sys.getsizeof(user)/1000000))
logger.info('here is an example record:')
logger.info('')
logger.info(user[0])

In [ ]:
del user

## review

In [ ]:
try:
    review = read_json('s3a://yelp-dataset-stevenhurwitt/yelp_academic_dataset_review.json')
    logger.info('read in json file')

except Exception as e:
    logger.warn(e)

OverflowError: signed integer is greater than maximum

In [ ]:
logger.info('json file has {} records with size of {} mb.'.format(len(review), sys.getsizeof(review)/1000000))
logger.info('here is an example record:')
logger.info('')
logger.info(review[0])

In [ ]:
del review